# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key  

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# 把在weather最後得到的data frame結果導入
data = "output_data/cities.csv"
file = pd.read_csv(data)
file

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Petropavlovsk-Kamchatskiy,53.0452,158.6483,6.80,66,30,6.71,RU,1610232878
1,Butaritari,3.0707,172.7902,81.63,81,50,20.27,KI,1610232878
2,Pouembout,-21.1333,164.9000,85.78,70,100,2.37,NC,1610232879
3,Port Alfred,-33.5906,26.8910,70.00,89,14,7.00,ZA,1610232880
4,Alyangula,-13.8483,136.4192,84.20,89,88,2.30,AU,1610232880
...,...,...,...,...,...,...,...,...,...
555,Nishihara,26.1842,127.7558,55.40,67,75,21.85,JP,1610233150
556,Tangier,35.7806,-5.8137,48.20,66,40,17.27,MA,1610233150
557,Bull Savanna,17.8868,-77.5902,74.62,84,89,5.26,JM,1610233151
558,Bogalusa,30.7910,-89.8487,48.20,49,1,9.22,US,1610233151


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
# 創一個heapmap顯示每個城市的 Humidity溼度, 用經緯度當locations, 用濕度當weight. 並加上heapmap layer
# 在這個步驟得到instrustions Part II - VacationPy 第一張照片

location = file[["Lat", "Lng"]]
humidity = file["Humidity"]


fig = gmaps.figure() 
heat = gmaps.heatmap_layer(location , weights = humidity  , dissipating=False, max_intensity=100,point_radius = 1)
fig.add_layer(heat)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# 縮小需要的資料的範圍  例如: 1) max temperature lower than 80 degrees but higher than 70.最高溫度在70-80中間(低於80   高於70)   
#  2) Wind speed less than 10 mph. 風速低於10mph   3) Zero cloudiness. 雲層為0  並只留下三個條件都符合的資料,只要有一項不符合就刪去

In [6]:
new = file.loc[(file["Max Temp"]>= 70) & (file["Max Temp"] <= 80), : ]

hotel_df = new.loc[(new["Wind Speed"] <= 10) & (new["Cloudiness"] == 0) , : ]

hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
38,Grenfell,-33.9000,148.1667,78.01,44,0,3.00,AU,1610232898
120,Paita,-5.0892,-81.1144,78.80,73,0,9.22,PE,1610232939
135,Lékoni,-1.5843,14.2591,74.59,73,0,5.99,GA,1610232946
158,Saint-Philippe,-21.3585,55.7679,73.40,78,0,4.61,RE,1610232957
183,Kidal,18.4411,1.4078,76.93,17,0,9.57,ML,1610232968
292,Saint-Joseph,-21.3667,55.6167,73.40,78,0,4.61,RE,1610233021
296,Pathein,16.7833,94.7333,72.12,83,0,3.65,MM,1610233024
314,Sawākin,19.1059,37.3321,75.58,75,0,9.84,SD,1610233032
371,Kolokani,13.5728,-8.0339,77.20,16,0,4.81,ML,1610232864
456,Agadez,19.7500,10.2500,70.25,22,0,7.20,NE,1610233101


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap. -->

In [7]:
# 步驟: 1. 先設一個variable 叫做 hotel 2. 然後在上面步驟得出的data frame裡加一欄叫做Hotel Name. 
# 3. 用Google Places API在每個城市裡找出在5000meters內的旅館 4.把API每個城市求出的第一個旅館存入hotel name那一欄
# 5. 在heapmap上用圖標 標示出那幾個hotel  並且每個圖標要包含 Hotel Name酒店名稱, City城市, and Country國家.
# 在這個步驟得到instrustions Part II - VacationPy 第2張照片

In [8]:
hotel_df["Hotel Name"] = ""
parametric = {"radius": 5000, "type": "lodging", "key": g_key}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    parametric["location"] = f"{lat},{lng}"
    
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    data = requests.get(url, params=parametric).json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = data["results"][0]["name"]
        
    except(KeyError, IndexError):
        print("missing giled...  skipping")

hotel_df.head()

C:\Users\sally\anaconda3\envs\pythondata\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\sally\anaconda3\envs\pythondata\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


missing giled...  skipping
missing giled...  skipping
missing giled...  skipping
missing giled...  skipping


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
38,Grenfell,-33.9000,148.1667,78.01,44,0,3.00,AU,1610232898,Grenfell Royal Hotel
120,Paita,-5.0892,-81.1144,78.80,73,0,9.22,PE,1610232939,Hotel Náutico de Paita
135,Lékoni,-1.5843,14.2591,74.59,73,0,5.99,GA,1610232946,Rendez-vous Des ChasseursD
158,Saint-Philippe,-21.3585,55.7679,73.40,78,0,4.61,RE,1610232957,"Chambres d'hôte ""La Trinité"""
183,Kidal,18.4411,1.4078,76.93,17,0,9.57,ML,1610232968,


In [9]:
namelist = hotel_df["Hotel Name"].tolist()
citylist = hotel_df["City"].tolist()
countrylist = hotel_df["Country"].tolist()

marker_location = hotel_df[["Lat","Lng"]]

info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""


marker = gmaps.marker_layer( marker_location, info_box_content=[info_box_template.format(**row) for index, row in hotel_df.iterrows()] )
fig.add_layer(marker)
fig.add_layer(heat)
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
#提示

# 用template 加圖標到heatmap上(不要改底下的code   照著打)
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""


# 把找到的旅館存到data frame裡
# 卻保有跟新 DataFrame 名稱
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]